In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import unittest
import tqdm.notebook as tq
import solitaire_env_hard as solitaire_env_hard
import sys
import seaborn as sns
import pickle as pkl
import gp
import tqdm.notebook as tq
#sys.setrecursionlimit(10**6)

In [10]:
class Node:
    def __init__(self,state=None):
        self.visits = 0
        self.children = []
        self.state = state
        self.action_scores = np.array([np.inf for _ in range(6)])
        self.parent = None
        self.child = None
        self.hashmap = {}
        
        self.visitted = False
        
        self.childCount = []
        
        self.possible_actions = []
    def won(self):
        
        pile_len = len(self.state.pile)
        
        if pile_len !=0:
            return False
        
        for i in range(7):
            if len(self.state.tableau[i]) != 0:
                return False
            
        for i in range(4):
            if len(self.state.tableau[i]) != 13:
                return False
            
        return True
    
    def stuck(self):
        
        test_env = solitaire_env_hard.env()
        test_env.state = self.state
        test_env.state = test_env.copy_state()
        
        
        for action in range(6):
            w,taken = test_env.step(action,fp_flag=True,debug=False)
            
            if taken == True:
                return False
            
            
        
        return True
    
    
    def find_possible_actions(self):
        
        possible_actions = []
        
        possible_tableau_to_foundation_reveal_actions = self.find_possible_tableau_to_foundation_reveal_actions()
        
        possible_pile_to_tableau_actions = self.find_possible_pile_to_tableau_actions()
        
        possible_to_foundation_stack_actions = self.find_possible_to_foundation_stack_actions()
        
        #possible_tableau_to_tableau_reveal_actions = self.find_possible_tableau_to_tableau_reveal_actions()
        
        #possible_tableau_to_tableau_not_reveal_actions = self.find_possible_tableau_to_tableau_not_reveal_actions()
        
        possible_foundation_to_tableau_actions = self.find_possible_foundation_to_tableau_actions()
        
        
        
        
        possible_actions.extend(possible_tableau_to_foundation_reveal_actions)
        
        possible_actions.extend(possible_pile_to_tableau_actions)
        
        possible_actions.extend(possible_to_foundation_stack_actions)
        
        possible_actions.extend(possible_foundation_to_tableau_actions)
        
        
        return possible_actions
      
    def find_possible_tableau_to_foundation_reveal_actions(self):
        
        moves = []
        
        for i in range(7):
            
            if len(self.state.tableau) <=2:
                continue
                
            if self.state.tableau[i][-2].face == 'up':
                continue
                
            cd = self.state.tableau[i][-1]
            
            suit = cd.get_suit_number()
            
            if cd.number == 1 or (self.state.foundation[suit].number == (cd.number-1)) :
                
                moves.append((0,i,cd,suit))
                
                
        return moves
    
    def find_possible_pile_to_tableau_actions(self):
        
        moves = []
        
        
        for i in range(len(self.state.pile)):
            
            if (i%3) == 0:
                cd1 = self.state.pile[i]
                
                for j in range(7):
                    
                    if len(self.state.tableau[j])>0:
                        
                        cd2 = self.state.tableau[j][-1]
                        
                        if cd1.color != cd2.color and cd1.number == (cd2.number-1):
                            moves.append((3,i,j))
                    
        return moves
    
    def default_policy_action(self):
        
        n = len(self.untried_actions)
        
        ind = random.sample(range(n),1)[0]
        
        action_index = self.untried_actions.pop(ind)
        
        return action_index, self.possible_actions[action_index]
    
    
    def best_action(self):
        
        ind = self.action_scores.argmax()
        
        
        return ind, self.possible_actions[action_index]
    
    
    def add_child(self,action,new_node):
        
        self.children[action[0]].append(new_node)
        
    
    def random_child(self,action):
        
        child = random.sample(self.children[action[0]],1)[0]
        
        return child
    
    def transition(self,action):
        
    def find_possible_to_foundation_stack_actions(self):
        
        moves = []
        
        for i in range(len(self.state.pile)):
            
            if (i%3) == 0:
                
                cd1 = self.state.pile[i]
                
                suit = cd1.get_suit_number()
                
                if cd1.number == 1:
                    
                    moves.append((2,0,i))
                    
                elif len(self.state.foundation[suit])>=1:
                    
                    cd2 = self.state.foundation[suit][-1]
                    
                    if cd1.number == (cd2.number+1):
                    
                        moves.append((2,0,i))
                    
                    
        for i in range(7):
            
            if len(self.state.tableau[i])>0:
                
                cd1 = self.state.tableau[i][-1] 
                suit = cd1.get_suit_number()
                
                if cd1.number == 1:
                    
                    moves.append((2,0,i))
                    
                elif len(self.state.foundation[suit])>=1:
                    
                    cd2 = self.state.foundation[suit][-1]
                    
                    if cd1.number == (cd2.number+1):
                    
                        moves.append((2,1,i))
                    
                    
        return moves
    
    
            
    
    def find_foundation_to_tableau_actions(self):
        
        moves = []
        
        for i in range(4):
            if len(self.state.foundation[i])>0:
                
                cd1 = self.state.foundation[i][-1]
                
                for j in range(7):
                    
                    if len(self.state.tableau[j])>0:
                        
                        cd2 = self.state.tableau[j][-1]
                        
                        if cd1.color != cd2.color and cd1.number == (cd2.number-1):
                            moves.append((4,cd1.get_suit_number(),j))
                            
                            
                            
        return moves

In [6]:
class MCTS:
    def __init__(self,c=0,d=100):
        self.c = c
        self.max_depth = d
        self.TreeMap = {}
        self.env = None
        
        self.current = env.state
     
        self.debug = [False for _ in range(6)]
    def isterminal(self):
        
        ans = (len(self.current.pile) == 0)
        
        #print(ans)
        for i in range(len(self.current.tableau)):
            ans = ans and (len(self.current.tableau[i]) == 0 )
            
            
        #print(ans)
        
        for i in range(len(self.current.foundation)):
            ans = ans and (len(self.current.foundation[i])==13)
            
            
        #print(ans)   
        return ans   
    
    def next_best_action(self):
            
        if max(self.current.action_scores) != np.inf:
            return self.current.action.argmax()


        for action in range(6):
            
            if self.current.action_scores[action] == np.inf:
                
                w = self.rollout(action)
                
                if w == "won":
                    print(w)
            
    def rollout(self):
        
        pass
    

In [10]:
env = solitaire_env_hard.env()

In [12]:
def update_scores(node,r):
        
        
        while self.parent is not None:
            
            parent = self.parent
            action = self.which_action_number
            
            parent.action_scores[action]+=r
            
            node = parent
            
            
            

In [11]:
number_of_trajectories = 100
sampling_width = 5

In [ ]:
node0 = Node()
node0.state = env.state


for i in range(number_of_trajectories):
    
    node = node0
    
    won = node.win() 
    
    stuck = node.stuck()
    
    while won == False and stuck == True:
        
        if node.visitted = False:
            
            possible_actions = node.find_possible_actions()
            
            node.visitted = True
            
            node.possible_actions = possible_actions
            node.number_of_possible_actions = len(possible_action)
            
            node.untried_actions = range(1,node.number_of_possible_actions+1)
            
            node.action_score = np.zeros(node.number_of_possible_actions)
            
            
        if len(node.untried_actions) == 0:
            
            action = node.best_action()
            
        else:
            action = node.default_policy_action()
            
            
        if node.childCount[action[0]] == sampling_width:
            new_node = node.random_child(action)
            
            
        else:
            next_node = node.transition(action)
            
            node.add_child(action,new_node)
            
            new_node.parent = node
            node.childCount[action[0]]+=1
            
            
        node = next_node
        
        
    
    update_scores(node,1 if won==True else 0)
        